In [1]:
import json
import pygraphviz as pgv
import networkx as nx

## Import JSON

In [2]:
f_output = open('mock_data/test_003.json')
data = json.load(f_output)

## Function Definition

In [3]:
# data["graph"]

## Extract Nodes and Edges

In [4]:
rule_node_ls=[]
edges_ls=[]

keys=data["graph"].keys()
for key in keys:
    rule=data["graph"][key]
    for dict_id in range(len(rule)):
        #rule level
        rule_node_ls.append(rule[dict_id]["name"])
        
        #body
        rule_body=rule[dict_id]["body"]
        for literal in rule_body:
            if literal["sign"] == False:
                edges_ls.append(("-"+literal["name"], rule[dict_id]["name"]))
            else:
                edges_ls.append((literal["name"], rule[dict_id]["name"]))
        #head 
        rule_head=rule[dict_id]["head"]
        if rule_head["sign"]==False:
            edges_ls.append((rule[dict_id]["name"],"-"+rule_head["name"]))
        else:
            edges_ls.append((rule[dict_id]["name"],rule_head["name"]))

In [5]:
edges_ls

[('a', 'r1'), ('b', 'r1'), ('r1', 'e'), ('d', 'r3'), ('e', 'r3'), ('r3', 'z')]

In [6]:
rule_node_ls

['r1', 'r3']

In [7]:
# for edge in edges_ls:
#     start_node=edge[0]
#     end_node=edge[1]
#     print('"{}"'.format(start_node)+" -> "+'"{}"'.format(end_node))

## Add Layer Information

In [8]:
node_layer={}

In [9]:
context=[]

for con_info in data["context"]:
    context.append(con_info["name"])
    node_layer[con_info["name"]]=0

In [10]:
node_layer

{'a': 0, 'b': 0, 'c': 0, 'd': 0}

In [11]:
rule_body={}
rule_head={}

for rule_id in rule_node_ls:
    rule_body_literals=[]
    for edge in edges_ls :
        if rule_id == edge[1]:
            rule_body_literals.append(edge[0])
        if rule_id ==edge[0]:
            rule_head_literals=edge[1]
        
    rule_body[rule_id]=rule_body_literals
    rule_head[rule_id]=rule_head_literals

In [12]:
rule_body

{'r1': ['a', 'b'], 'r3': ['d', 'e']}

In [13]:
rule_head

{'r1': 'e', 'r3': 'z'}

In [14]:
def layer_assign(rule_node_ls, rule_body, rule_head, node_layer):
    if len(rule_node_ls)!=0:
        for rule_id in rule_node_ls:
            rule_body_literals=rule_body[rule_id]
            assigned_nodes=list(node_layer.keys())
            is_subset = set(rule_body_literals).issubset(set(assigned_nodes))
            if is_subset==True:
                layer_info=[node_layer[key] for key in rule_body_literals]
                node_layer[rule_id]=max(layer_info)+1
                node_layer[rule_head[rule_id]]=max(layer_info)+2
                rule_node_ls.remove(rule_id)
        layer_assign(rule_node_ls, rule_body, rule_head, node_layer)
    else:
        return(node_layer)

In [15]:
layer_assign(rule_node_ls, rule_body, rule_head, node_layer)

In [16]:
node_layer

{'a': 0, 'b': 0, 'c': 0, 'd': 0, 'r1': 1, 'e': 2, 'r3': 3, 'z': 4}

## Graph Construction

In [17]:
G = nx.DiGraph(rankdir="LR")

## add node
for node in list(node_layer.keys()):
    if node[0]=="R" or node[0]=="r":
        G.add_node(node, shape="box",style="rounded")
    else:
        if node[0]=="-":
            G.add_node(node[1:], color="#ff6262",style="filled")
        else:
            G.add_node(node, color="#a2e665",style="filled")

#add edge
for edge in edges_ls:
    if edge[0][0]=="-":  # negation to rule
        G.add_edge(edge[0][1:], edge[1], color="#ff6262")
    elif edge[0][0]!="R" or edge[0][0]!="r": # positive literal to rule
        G.add_edge(edge[0], edge[1],color="#a2e665")
    else: # rule to literal
        if edge[1][0]=="-":
            G.add_edge(edge[0], edge[1][1:],color="#ff6262")
        else:
            G.add_edge(edge[0], edge[1],color="#a2e665")

#add rank
A = nx.nx_agraph.to_agraph(G)

for layer_id in set(node_layer.values()):
    node_ls=[node for node,layer in node_layer.items() if layer ==layer_id]
    A.add_subgraph(node_ls,rank='same')

A.draw("file.png",prog="dot")